# 과제 - 4주차

## 데이터 포맷에 적합한 elasticsearch index 생성하기

### 기본코드

In [ ]:
import argparse
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import sys
sys.path.append('/home/jovyan/jobs')
from base import read_input, init_df, df_with_meta
from filter import DailyStatFilter, TopRepoFilter, TopUserFilter, PytorchTopIssuerFilter
from es import Es

# SparkSession
spark = (SparkSession
    .builder
    .master("local")
    .appName("spark-sql")
    .config("spark.driver.extraClassPath", "/opt/bitnami/spark/resources/elasticsearch-spark-30_2.12-8.4.3.jar")
    .config("spark.jars", "/opt/bitnami/spark/resources/elasticsearch-spark-30_2.12-8.4.3.jar")
    # for jupyter
    .config("spark.driver.extraClassPath", "/home/jovyan/resources/elasticsearch-spark-30_2.12-8.4.3.jar")
    .config("spark.jars", "/home/jovyan/resources/elasticsearch-spark-30_2.12-8.4.3.jar")   
    # 옵션추가 시작
    .config("spark.executor.memory","3G")
    .config("spark.driver.memory","3G")
    .config("num-executor",2)
    .config("executor-cores",2)
    # 옵션추가 끝
    .getOrCreate())

class Args:
    def __init__(self):
        self.target_date = None
        self.input_path = None
        self.spark = None

args = Args()
args.spark = spark

In [5]:
# for Jupyter test
args.input_path = f"/home/jovyan/data/gh_archive/2024-08-24-*.json.gz"

### 실습코드 데이터 확인

In [6]:
df = read_input(args.spark, args.input_path)
df = init_df(df)

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

In [28]:
# DailyStatFilter 산출데이터 확인
stat_filter = DailyStatFilter(args)
stat_df = stat_filter.filter(df)
stat_df.show()

24/09/02 14:19:27 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/02 14:19:27 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/02 14:19:27 INFO FileSourceStrategy: Pushed Filters: 
24/09/02 14:19:27 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/02 14:19:27 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
24/09/02 14:19:27 INFO MemoryStore: Block broadcast_78 stored as values in memory (estimated size 176.4 KiB, free 1661.4 MiB)
24/09/02 14:19:27 INFO MemoryStore: Block broadcast_78_piece0 stored as bytes in memory (estimated size 28.1 KiB, free 1661.4 MiB)
24/09/02 14:19:27 INFO BlockManagerInfo: Added broadcast_78_piece0 in memory on 142a3f268980:46091 (size: 28.1 KiB, free: 1662.9 MiB)
24/09/02 14:19:27 INFO SparkContext: Created broadcast 78 from json at NativeMethodAccessorImpl.java:0
24/09/02 14:19:27 INFO FileSourceScanExec: Planning scan with bin packing, max size: 60559581 bytes, open cost is considered

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/09/02 14:19:31 WARN CacheManager: Asked to cache already cached data.
24/09/02 14:19:31 WARN CacheManager: Asked to cache already cached data.
24/09/02 14:19:31 WARN CacheManager: Asked to cache already cached data.
24/09/02 14:19:31 WARN CacheManager: Asked to cache already cached data.
24/09/02 14:19:31 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:19:31 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:19:31 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>>
24/09/02 14:19:31 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:19:31 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:19:31 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, displ

24/09/02 14:19:35 INFO Executor: Finished task 0.0 in stage 66.0 (TID 1642). 2741 bytes result sent to driver
24/09/02 14:19:35 INFO TaskSetManager: Finished task 0.0 in stage 66.0 (TID 1642) in 3512 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:19:35 INFO TaskSchedulerImpl: Removed TaskSet 66.0, whose tasks have all completed, from pool 
24/09/02 14:19:35 INFO DAGScheduler: ShuffleMapStage 66 ($anonfun$withThreadLocalCaptured$1 at FutureTask.java:264) finished in 3.514 s
24/09/02 14:19:35 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:19:35 INFO DAGScheduler: running: Set()
24/09/02 14:19:35 INFO DAGScheduler: waiting: Set(ShuffleMapStage 67, ResultStage 68)
24/09/02 14:19:35 INFO DAGScheduler: failed: Set()
24/09/02 14:19:35 INFO DAGScheduler: Submitting ShuffleMapStage 67 (MapPartitionsRDD[280] at $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264), which has no missing parents
24/09/02 14:19:35 INFO MemoryStore: Block broadcast_90 stored as values 

24/09/02 14:19:35 INFO ShuffleBlockFetcherIterator: Getting 1 (955.0 B) non-empty blocks including 1 (955.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:35 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:35 INFO Executor: Finished task 40.0 in stage 67.0 (TID 1683). 4181 bytes result sent to driver
24/09/02 14:19:35 INFO TaskSetManager: Starting task 41.0 in stage 67.0 (TID 1684) (142a3f268980, executor driver, partition 41, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:35 INFO TaskSetManager: Finished task 40.0 in stage 67.0 (TID 1683) in 5 ms on 142a3f268980 (executor driver) (41/200)
24/09/02 14:19:35 INFO Executor: Running task 41.0 in stage 67.0 (TID 1684)
24/09/02 14:19:35 INFO ShuffleBlockFetcherIterator: Getting 1 (1156.0 B) non-empty blocks including 1 (1156.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:35 INFO ShuffleBlockFetcherIterator: Started 0 remote 

...

24/09/02 14:19:43 INFO DAGScheduler: ShuffleMapStage 81 ($anonfun$withThreadLocalCaptured$1 at FutureTask.java:264) finished in 0.802 s
24/09/02 14:19:43 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:19:43 INFO DAGScheduler: running: Set()
24/09/02 14:19:43 INFO DAGScheduler: waiting: Set(ResultStage 82)
24/09/02 14:19:43 INFO DAGScheduler: failed: Set()
24/09/02 14:19:43 INFO DAGScheduler: Submitting ResultStage 82 (MapPartitionsRDD[336] at $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264), which has no missing parents
24/09/02 14:19:43 INFO MemoryStore: Block broadcast_108 stored as values in memory (estimated size 10.1 KiB, free 1662.1 MiB)
24/09/02 14:19:43 INFO MemoryStore: Block broadcast_108_piece0 stored as bytes in memory (estimated size 5.0 KiB, free 1662.1 MiB)
24/09/02 14:19:43 INFO BlockManagerInfo: Added broadcast_108_piece0 in memory on 142a3f268980:46091 (size: 5.0 KiB, free: 1663.0 MiB)
24/09/02 14:19:43 INFO SparkContext: Created broadcast 

24/09/02 14:19:45 INFO Executor: Finished task 61.0 in stage 84.0 (TID 2314). 4181 bytes result sent to driver
24/09/02 14:19:45 INFO TaskSetManager: Starting task 62.0 in stage 84.0 (TID 2315) (142a3f268980, executor driver, partition 62, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:45 INFO TaskSetManager: Finished task 61.0 in stage 84.0 (TID 2314) in 3 ms on 142a3f268980 (executor driver) (62/200)
24/09/02 14:19:45 INFO Executor: Running task 62.0 in stage 84.0 (TID 2315)
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Getting 1 (3.9 KiB) non-empty blocks including 1 (3.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:45 INFO Executor: Finished task 62.0 in stage 84.0 (TID 2315). 4181 bytes result sent to driver
24/09/02 14:19:45 INFO TaskSetManager: Starting task 63.0 in stage 84.0 (TID 2316) (142a3f268980, executor driver, partition 63, N

24/09/02 14:19:45 INFO Executor: Finished task 119.0 in stage 84.0 (TID 2372). 4181 bytes result sent to driver
24/09/02 14:19:45 INFO TaskSetManager: Starting task 120.0 in stage 84.0 (TID 2373) (142a3f268980, executor driver, partition 120, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:45 INFO TaskSetManager: Finished task 119.0 in stage 84.0 (TID 2372) in 5 ms on 142a3f268980 (executor driver) (120/200)
24/09/02 14:19:45 INFO Executor: Running task 120.0 in stage 84.0 (TID 2373)
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Getting 1 (3.2 KiB) non-empty blocks including 1 (3.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:45 INFO Executor: Finished task 120.0 in stage 84.0 (TID 2373). 4181 bytes result sent to driver
24/09/02 14:19:45 INFO TaskSetManager: Starting task 121.0 in stage 84.0 (TID 2374) (142a3f268980, executor driver, partiti

+------------+------------+----------+--------+----------+--------------------+
|d_user_count|d_repo_count|push_count|pr_count|fork_count|commit_comment_count|
+------------+------------+----------+--------+----------+--------------------+
|       37540|       27844|     74882|    7573|      1685|                  89|
+------------+------------+----------+--------+----------+--------------------+



24/09/02 14:19:45 INFO TaskSetManager: Starting task 172.0 in stage 84.0 (TID 2425) (142a3f268980, executor driver, partition 172, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:45 INFO TaskSetManager: Finished task 171.0 in stage 84.0 (TID 2424) in 4 ms on 142a3f268980 (executor driver) (172/200)
24/09/02 14:19:45 INFO Executor: Running task 172.0 in stage 84.0 (TID 2425)
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Getting 1 (3.2 KiB) non-empty blocks including 1 (3.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:45 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:45 INFO Executor: Finished task 172.0 in stage 84.0 (TID 2425). 4181 bytes result sent to driver
24/09/02 14:19:45 INFO TaskSetManager: Starting task 173.0 in stage 84.0 (TID 2426) (142a3f268980, executor driver, partition 173, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:45 INFO TaskSetManager: Finished ta

In [29]:
# DailyStatFilter 산출데이터 with metadata(timestamp) 확인
stat_df = df_with_meta(stat_df, args.target_date)
stat_df.show()

24/09/02 14:21:13 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:21:13 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:21:13 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>>
24/09/02 14:21:13 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:21:13 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:21:13 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, payload: struct<action: string, before: string, comment: struct<_links: struct<html: struct<href: string>, pull_request: struct<href: string>, self: struct<href: string> ... 1 more fields>, author_association

24/09/02 14:21:14 INFO BlockManagerInfo: Removed broadcast_118_piece0 on 142a3f268980:46091 in memory (size: 8.8 KiB, free: 1663.0 MiB)
24/09/02 14:21:17 INFO Executor: Finished task 0.0 in stage 94.0 (TID 2458). 2741 bytes result sent to driver
24/09/02 14:21:17 INFO TaskSetManager: Finished task 0.0 in stage 94.0 (TID 2458) in 3396 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:21:17 INFO TaskSchedulerImpl: Removed TaskSet 94.0, whose tasks have all completed, from pool 
24/09/02 14:21:17 INFO DAGScheduler: ShuffleMapStage 94 ($anonfun$withThreadLocalCaptured$1 at FutureTask.java:264) finished in 3.400 s
24/09/02 14:21:17 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:21:17 INFO DAGScheduler: running: Set()
24/09/02 14:21:17 INFO DAGScheduler: waiting: Set(ResultStage 96, ShuffleMapStage 95)
24/09/02 14:21:17 INFO DAGScheduler: failed: Set()
24/09/02 14:21:17 INFO DAGScheduler: Submitting ShuffleMapStage 95 (MapPartitionsRDD[386] at $anonfun$withThreadLocalC

24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (955.0 B) non-empty blocks including 1 (955.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:17 INFO Executor: Finished task 43.0 in stage 95.0 (TID 2502). 4181 bytes result sent to driver
24/09/02 14:21:17 INFO TaskSetManager: Starting task 44.0 in stage 95.0 (TID 2503) (142a3f268980, executor driver, partition 44, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:17 INFO TaskSetManager: Finished task 43.0 in stage 95.0 (TID 2502) in 5 ms on 142a3f268980 (executor driver) (44/200)
24/09/02 14:21:17 INFO Executor: Running task 44.0 in stage 95.0 (TID 2503)
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (868.0 B) non-empty blocks including 1 (868.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fe

24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (955.0 B) non-empty blocks including 1 (955.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:17 INFO Executor: Finished task 90.0 in stage 95.0 (TID 2549). 4181 bytes result sent to driver
24/09/02 14:21:17 INFO TaskSetManager: Starting task 91.0 in stage 95.0 (TID 2550) (142a3f268980, executor driver, partition 91, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:17 INFO TaskSetManager: Finished task 90.0 in stage 95.0 (TID 2549) in 7 ms on 142a3f268980 (executor driver) (91/200)
24/09/02 14:21:17 INFO Executor: Running task 91.0 in stage 95.0 (TID 2550)
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (955.0 B) non-empty blocks including 1 (955.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fe

24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (955.0 B) non-empty blocks including 1 (955.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:17 INFO Executor: Finished task 137.0 in stage 95.0 (TID 2596). 4181 bytes result sent to driver
24/09/02 14:21:17 INFO TaskSetManager: Starting task 138.0 in stage 95.0 (TID 2597) (142a3f268980, executor driver, partition 138, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:17 INFO TaskSetManager: Finished task 137.0 in stage 95.0 (TID 2596) in 4 ms on 142a3f268980 (executor driver) (138/200)
24/09/02 14:21:17 INFO Executor: Running task 138.0 in stage 95.0 (TID 2597)
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Getting 1 (868.0 B) non-empty blocks including 1 (868.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:17 INFO ShuffleBlockFetcherIterator: Started 0 rem

...

24/09/02 14:21:19 INFO BlockManagerInfo: Removed broadcast_124_piece0 on 142a3f268980:46091 in memory (size: 27.3 KiB, free: 1663.0 MiB)
24/09/02 14:21:19 INFO BlockManagerInfo: Removed broadcast_125_piece0 on 142a3f268980:46091 in memory (size: 5.0 KiB, free: 1663.0 MiB)
24/09/02 14:21:19 INFO Executor: Finished task 0.0 in stage 97.0 (TID 2660). 2741 bytes result sent to driver
24/09/02 14:21:19 INFO TaskSetManager: Finished task 0.0 in stage 97.0 (TID 2660) in 1793 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:21:19 INFO TaskSchedulerImpl: Removed TaskSet 97.0, whose tasks have all completed, from pool 
24/09/02 14:21:19 INFO DAGScheduler: ShuffleMapStage 97 (showString at NativeMethodAccessorImpl.java:0) finished in 1.795 s
24/09/02 14:21:19 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:21:19 INFO DAGScheduler: running: Set()
24/09/02 14:21:19 INFO DAGScheduler: waiting: Set(ResultStage 99, ShuffleMapStage 98)
24/09/02 14:21:19 INFO DAGScheduler: failed:

24/09/02 14:21:20 INFO Executor: Finished task 55.0 in stage 98.0 (TID 2716). 4181 bytes result sent to driver
24/09/02 14:21:20 INFO TaskSetManager: Starting task 56.0 in stage 98.0 (TID 2717) (142a3f268980, executor driver, partition 56, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:20 INFO TaskSetManager: Finished task 55.0 in stage 98.0 (TID 2716) in 4 ms on 142a3f268980 (executor driver) (56/200)
24/09/02 14:21:20 INFO Executor: Running task 56.0 in stage 98.0 (TID 2717)
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Getting 1 (2.9 KiB) non-empty blocks including 1 (2.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:20 INFO Executor: Finished task 56.0 in stage 98.0 (TID 2717). 4181 bytes result sent to driver
24/09/02 14:21:20 INFO TaskSetManager: Starting task 57.0 in stage 98.0 (TID 2718) (142a3f268980, executor driver, partition 57, N

24/09/02 14:21:20 INFO Executor: Finished task 111.0 in stage 98.0 (TID 2772). 4181 bytes result sent to driver
24/09/02 14:21:20 INFO TaskSetManager: Starting task 112.0 in stage 98.0 (TID 2773) (142a3f268980, executor driver, partition 112, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:20 INFO TaskSetManager: Finished task 111.0 in stage 98.0 (TID 2772) in 4 ms on 142a3f268980 (executor driver) (112/200)
24/09/02 14:21:20 INFO Executor: Running task 112.0 in stage 98.0 (TID 2773)
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Getting 1 (3.5 KiB) non-empty blocks including 1 (3.5 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:20 INFO Executor: Finished task 112.0 in stage 98.0 (TID 2773). 4181 bytes result sent to driver
24/09/02 14:21:20 INFO TaskSetManager: Starting task 113.0 in stage 98.0 (TID 2774) (142a3f268980, executor driver, partiti

+------------+------------+----------+--------+----------+--------------------+----------+
|d_user_count|d_repo_count|push_count|pr_count|fork_count|commit_comment_count|@timestamp|
+------------+------------+----------+--------+----------+--------------------+----------+
|       37540|       27844|     74882|    7573|      1685|                  89|2024-09-01|
+------------+------------+----------+--------+----------+--------------------+----------+



24/09/02 14:21:20 INFO TaskSetManager: Starting task 173.0 in stage 98.0 (TID 2834) (142a3f268980, executor driver, partition 173, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:20 INFO TaskSetManager: Finished task 172.0 in stage 98.0 (TID 2833) in 4 ms on 142a3f268980 (executor driver) (173/200)
24/09/02 14:21:20 INFO Executor: Running task 173.0 in stage 98.0 (TID 2834)
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Getting 1 (3.5 KiB) non-empty blocks including 1 (3.5 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:21:20 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:21:20 INFO Executor: Finished task 173.0 in stage 98.0 (TID 2834). 4181 bytes result sent to driver
24/09/02 14:21:20 INFO TaskSetManager: Starting task 174.0 in stage 98.0 (TID 2835) (142a3f268980, executor driver, partition 174, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:21:20 INFO TaskSetManager: Finished ta

In [7]:
stat_df.show(truncate=False, vertical=False)

NameError: name 'stat_df' is not defined

### eleasticsearch index생성할 데이터 확인

In [3]:
import pyspark.sql.functions as F

base_df = df.filter(F.col('userid_and_repo_name') == 'pytorch/pytorch')
base_df.show()

+-----------------+--------------------+-------------------+-----------+-------------+----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|        user_name|                 url|         created_at|         id|repository_id|size|distinct_size|             comment|                type|            repo_url|userid_and_repo_name|repo_name|
+-----------------+--------------------+-------------------+-----------+-------------+----+-------------+--------------------+--------------------+--------------------+--------------------+---------+
|    SherlockNoMad|https://api.githu...|2024-08-24 10:00:11|41304524101|         null|null|         null|                null|PullRequestReview...|https://api.githu...|     pytorch/pytorch|  pytorch|
| pytorch-bot[bot]|https://api.githu...|2024-08-24 10:00:19|41304525537|         null|null|         null|                null|         CreateEvent|https://api.githu...|     pytorch/pytorch|  pytorch|


In [34]:
result_df = base_df.groupBy('user_name').pivot('type').count()
result_df.show()

24/09/02 14:25:12 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor),IsNotNull(repo)
24/09/02 14:25:12 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),isnotnull(repo#841),NOT (actor#835.login = github-actions[bot]),(repo#841.name = pytorch/pytorch)
24/09/02 14:25:12 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, repo: struct<id: bigint, name: string, url: string ... 1 more fields>, type: string ... 1 more fields>
24/09/02 14:25:12 INFO CodeGenerator: Code generated in 5.6585 ms
24/09/02 14:25:12 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/09/02 14:25:12 INFO CodeGenerator: Code generated in 6.8594 ms
24/09/02 14:25:12 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but cu

24/09/02 14:25:14 INFO Executor: Finished task 60.0 in stage 105.0 (TID 3131). 3742 bytes result sent to driver
24/09/02 14:25:14 INFO TaskSetManager: Starting task 61.0 in stage 105.0 (TID 3132) (142a3f268980, executor driver, partition 61, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:25:14 INFO TaskSetManager: Finished task 60.0 in stage 105.0 (TID 3131) in 4 ms on 142a3f268980 (executor driver) (66/200)
24/09/02 14:25:14 INFO Executor: Running task 61.0 in stage 105.0 (TID 3132)
24/09/02 14:25:14 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:14 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:25:14 INFO Executor: Finished task 61.0 in stage 105.0 (TID 3132). 3742 bytes result sent to driver
24/09/02 14:25:14 INFO TaskSetManager: Starting task 62.0 in stage 105.0 (TID 3133) (142a3f268980, executor driver, partition 

24/09/02 14:25:14 INFO Executor: Finished task 136.0 in stage 105.0 (TID 3204). 3656 bytes result sent to driver
24/09/02 14:25:14 INFO TaskSetManager: Starting task 137.0 in stage 105.0 (TID 3205) (142a3f268980, executor driver, partition 137, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:25:14 INFO TaskSetManager: Finished task 136.0 in stage 105.0 (TID 3204) in 3 ms on 142a3f268980 (executor driver) (139/200)
24/09/02 14:25:14 INFO Executor: Running task 137.0 in stage 105.0 (TID 3205)
24/09/02 14:25:14 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:14 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:25:14 INFO Executor: Finished task 137.0 in stage 105.0 (TID 3205). 3656 bytes result sent to driver
24/09/02 14:25:14 INFO TaskSetManager: Starting task 138.0 in stage 105.0 (TID 3206) (142a3f268980, executor driver, pa

24/09/02 14:25:14 INFO CodeGenerator: Code generated in 19.4349 ms
24/09/02 14:25:14 INFO MemoryStore: Block broadcast_141 stored as values in memory (estimated size 176.3 KiB, free 1662.0 MiB)
24/09/02 14:25:14 INFO MemoryStore: Block broadcast_141_piece0 stored as bytes in memory (estimated size 28.0 KiB, free 1662.0 MiB)
24/09/02 14:25:14 INFO BlockManagerInfo: Added broadcast_141_piece0 in memory on 142a3f268980:46091 (size: 28.0 KiB, free: 1663.0 MiB)
24/09/02 14:25:14 INFO SparkContext: Created broadcast 141 from showString at NativeMethodAccessorImpl.java:0
24/09/02 14:25:14 INFO FileSourceScanExec: Planning scan with bin packing, max size: 60559581 bytes, open cost is considered as scanning 4194304 bytes.
24/09/02 14:25:14 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/02 14:25:14 INFO DAGScheduler: Registering RDD 441 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 36
24/09/02 14:25:14 INFO DAGScheduler: Registering RDD

24/09/02 14:25:16 INFO Executor: Finished task 192.0 in stage 108.0 (TID 3275). 4383 bytes result sent to driver
24/09/02 14:25:16 INFO TaskSetManager: Starting task 0.0 in stage 108.0 (TID 3276) (142a3f268980, executor driver, partition 0, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:25:16 INFO TaskSetManager: Finished task 192.0 in stage 108.0 (TID 3275) in 10 ms on 142a3f268980 (executor driver) (8/200)
24/09/02 14:25:16 INFO Executor: Running task 0.0 in stage 108.0 (TID 3276)
24/09/02 14:25:16 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:16 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:25:16 INFO Executor: Finished task 0.0 in stage 108.0 (TID 3276). 4254 bytes result sent to driver
24/09/02 14:25:16 INFO TaskSetManager: Starting task 1.0 in stage 108.0 (TID 3277) (142a3f268980, executor driver, partition 1, 

...

24/09/02 14:25:17 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/02 14:25:17 INFO DAGScheduler: Got job 48 (showString at NativeMethodAccessorImpl.java:0) with 4 output partitions
24/09/02 14:25:17 INFO DAGScheduler: Final stage: ResultStage 112 (showString at NativeMethodAccessorImpl.java:0)
24/09/02 14:25:17 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 111)
24/09/02 14:25:17 INFO DAGScheduler: Missing parents: List()
24/09/02 14:25:17 INFO DAGScheduler: Submitting ResultStage 112 (MapPartitionsRDD[454] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/09/02 14:25:17 INFO MemoryStore: Block broadcast_145 stored as values in memory (estimated size 45.9 KiB, free 1662.0 MiB)
24/09/02 14:25:17 INFO MemoryStore: Block broadcast_145_piece0 stored as bytes in memory (estimated size 19.6 KiB, free 1662.0 MiB)
24/09/02 14:25:17 INFO BlockManagerInfo: Added broadcast_145_piece0 in memory on 142a3f268980:4

24/09/02 14:25:17 INFO MemoryStore: Block rdd_449_40 stored as values in memory (estimated size 16.0 B, free 1662.1 MiB)
24/09/02 14:25:17 INFO BlockManagerInfo: Added rdd_449_40 in memory on 142a3f268980:46091 (size: 16.0 B, free: 1663.0 MiB)
24/09/02 14:25:17 INFO Executor: Finished task 15.0 in stage 118.0 (TID 3510). 5069 bytes result sent to driver
24/09/02 14:25:17 INFO TaskSetManager: Starting task 16.0 in stage 118.0 (TID 3511) (142a3f268980, executor driver, partition 41, PROCESS_LOCAL, 4453 bytes) taskResourceAssignments Map()
24/09/02 14:25:17 INFO TaskSetManager: Finished task 15.0 in stage 118.0 (TID 3510) in 4 ms on 142a3f268980 (executor driver) (18/100)
24/09/02 14:25:17 INFO Executor: Running task 16.0 in stage 118.0 (TID 3511)
24/09/02 14:25:17 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in

24/09/02 14:25:18 INFO MemoryStore: Block rdd_449_102 stored as values in memory (estimated size 16.0 B, free 1662.1 MiB)
24/09/02 14:25:18 INFO BlockManagerInfo: Added rdd_449_102 in memory on 142a3f268980:46091 (size: 16.0 B, free: 1663.0 MiB)
24/09/02 14:25:18 INFO Executor: Finished task 77.0 in stage 118.0 (TID 3571). 5069 bytes result sent to driver
24/09/02 14:25:18 INFO TaskSetManager: Starting task 78.0 in stage 118.0 (TID 3572) (142a3f268980, executor driver, partition 103, PROCESS_LOCAL, 4453 bytes) taskResourceAssignments Map()
24/09/02 14:25:18 INFO TaskSetManager: Finished task 77.0 in stage 118.0 (TID 3571) in 4 ms on 142a3f268980 (executor driver) (79/100)
24/09/02 14:25:18 INFO Executor: Running task 78.0 in stage 118.0 (TID 3572)
24/09/02 14:25:18 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:18 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches

+-----------------+-----------+-----------+-----------------+----------------+----------------------+----------+
|        user_name|CreateEvent|DeleteEvent|IssueCommentEvent|PullRequestEvent|PullRequestReviewEvent|WatchEvent|
+-----------------+-----------+-----------+-----------------+----------------+----------------------+----------+
| pytorch-bot[bot]|          4|          2|                1|            null|                  null|      null|
|leslie-fang-intel|       null|       null|                1|            null|                     2|      null|
|cybersecurity-dev|       null|       null|             null|            null|                  null|         1|
|          xuhancn|       null|       null|             null|               1|                  null|      null|
|    SherlockNoMad|       null|       null|             null|            null|                     1|      null|
+-----------------+-----------+-----------+-----------------+----------------+------------------

24/09/02 14:25:18 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:25:18 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:25:18 INFO MemoryStore: Block rdd_449_159 stored as values in memory (estimated size 16.0 B, free 1662.0 MiB)
24/09/02 14:25:18 INFO BlockManagerInfo: Added rdd_449_159 in memory on 142a3f268980:46091 (size: 16.0 B, free: 1663.0 MiB)
24/09/02 14:25:18 INFO Executor: Finished task 34.0 in stage 121.0 (TID 3628). 5069 bytes result sent to driver
24/09/02 14:25:18 INFO TaskSetManager: Starting task 35.0 in stage 121.0 (TID 3629) (142a3f268980, executor driver, partition 160, PROCESS_LOCAL, 4453 bytes) taskResourceAssignments Map()
24/09/02 14:25:18 INFO TaskSetManager: Finished task 34.0 in stage 121.0 (TID 3628) in 4 ms on 142a3f268980 (executor driver) (36/75)
24/09/02 14:25:18 INFO Executor: Running task 35.0 in stage 121.0 (TI

In [35]:
issues_event_exists = base_df.filter(df["type"] == "IssuesEvent").count() > 0
issues_event_exists

24/09/02 14:26:08 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor),IsNotNull(repo),IsNotNull(type),EqualTo(type,IssuesEvent)
24/09/02 14:26:08 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),isnotnull(repo#841),isnotnull(type#842),NOT (actor#835.login = github-actions[bot]),(repo#841.name = pytorch/pytorch),(type#842 = IssuesEvent)
24/09/02 14:26:08 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, repo: struct<id: bigint, name: string, url: string ... 1 more fields>, type: string ... 1 more fields>
24/09/02 14:26:08 INFO CodeGenerator: Code generated in 4.5285 ms
24/09/02 14:26:08 INFO CodeGenerator: Code generated in 4.935301 ms
24/09/02 14:26:08 INFO MemoryStore: Block broadcast_149 stored as values in memory (estimated size 176.3 KiB, free 1661.8 MiB)
24/09/02 14:26:08 INFO MemoryStore: Block broadcast_149_piece0 stored as bytes in m

False

In [38]:
result_df = base_df.groupBy('user_name').pivot('type').count()
result_df = result_df.cache()
result_df.where((~F.col('user_name').contains('[bot]'))) \
            .orderBy(F.desc('CreateEvent')) \
            .limit(10).show()


24/09/02 14:27:36 INFO BlockManagerInfo: Removed broadcast_155_piece0 on 142a3f268980:46091 in memory (size: 15.0 KiB, free: 1663.0 MiB)
24/09/02 14:27:36 INFO BlockManagerInfo: Removed broadcast_154_piece0 on 142a3f268980:46091 in memory (size: 14.9 KiB, free: 1663.0 MiB)
24/09/02 14:27:36 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor),IsNotNull(repo)
24/09/02 14:27:36 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),isnotnull(repo#841),NOT (actor#835.login = github-actions[bot]),(repo#841.name = pytorch/pytorch)
24/09/02 14:27:36 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, repo: struct<id: bigint, name: string, url: string ... 1 more fields>, type: string ... 1 more fields>
24/09/02 14:27:36 INFO CodeGenerator: Code generated in 4.3936 ms
24/09/02 14:27:36 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled 

24/09/02 14:27:38 INFO TaskSetManager: Starting task 61.0 in stage 128.0 (TID 3939) (142a3f268980, executor driver, partition 61, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:38 INFO TaskSetManager: Finished task 60.0 in stage 128.0 (TID 3938) in 3 ms on 142a3f268980 (executor driver) (66/200)
24/09/02 14:27:38 INFO Executor: Running task 61.0 in stage 128.0 (TID 3939)
24/09/02 14:27:38 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:27:38 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:27:38 INFO Executor: Finished task 61.0 in stage 128.0 (TID 3939). 3656 bytes result sent to driver
24/09/02 14:27:38 INFO TaskSetManager: Starting task 62.0 in stage 128.0 (TID 3940) (142a3f268980, executor driver, partition 62, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:38 INFO TaskSetManager: Finished tas

24/09/02 14:27:38 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:27:38 INFO Executor: Finished task 147.0 in stage 128.0 (TID 4022). 3656 bytes result sent to driver
24/09/02 14:27:38 INFO TaskSetManager: Starting task 148.0 in stage 128.0 (TID 4023) (142a3f268980, executor driver, partition 148, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:38 INFO TaskSetManager: Finished task 147.0 in stage 128.0 (TID 4022) in 2 ms on 142a3f268980 (executor driver) (150/200)
24/09/02 14:27:38 INFO Executor: Running task 148.0 in stage 128.0 (TID 4023)
24/09/02 14:27:38 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:27:38 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:27:38 INFO Executor: Finished task 148.0 in stage 128.0 (TID 4023). 3656 bytes result sent to driver
24/09/02 14:27:38 INFO TaskSetMana

24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_0 locally
24/09/02 14:27:39 INFO CodeGenerator: Code generated in 2.5822 ms
24/09/02 14:27:39 INFO CodeGenerator: Code generated in 6.8093 ms
24/09/02 14:27:39 INFO Executor: Finished task 0.0 in stage 132.0 (TID 4074). 5387 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 1.0 in stage 132.0 (TID 4075) (142a3f268980, executor driver, partition 1, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Finished task 0.0 in stage 132.0 (TID 4074) in 36 ms on 142a3f268980 (executor driver) (1/200)
24/09/02 14:27:39 INFO Executor: Running task 1.0 in stage 132.0 (TID 4075)
24/09/02 14:27:39 INFO BlockManagerInfo: Removed broadcast_156_piece0 on 142a3f268980:46091 in memory (size: 28.0 KiB, free: 1663.0 MiB)
24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_1 locally
24/09/02 14:27:39 INFO Executor: Finished task 1.0 in stage 132.0 (TID 4075). 5344 bytes re

24/09/02 14:27:39 INFO Executor: Finished task 53.0 in stage 132.0 (TID 4127). 5387 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 54.0 in stage 132.0 (TID 4128) (142a3f268980, executor driver, partition 54, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Finished task 53.0 in stage 132.0 (TID 4127) in 4 ms on 142a3f268980 (executor driver) (54/200)
24/09/02 14:27:39 INFO Executor: Running task 54.0 in stage 132.0 (TID 4128)
24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_54 locally
24/09/02 14:27:39 INFO Executor: Finished task 54.0 in stage 132.0 (TID 4128). 5344 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 55.0 in stage 132.0 (TID 4129) (142a3f268980, executor driver, partition 55, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Finished task 54.0 in stage 132.0 (TID 4128) in 4 ms on 142a3f268980 (executor driv

24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_114 locally
24/09/02 14:27:39 INFO Executor: Finished task 114.0 in stage 132.0 (TID 4188). 5344 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 115.0 in stage 132.0 (TID 4189) (142a3f268980, executor driver, partition 115, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO Executor: Running task 115.0 in stage 132.0 (TID 4189)
24/09/02 14:27:39 INFO TaskSetManager: Finished task 114.0 in stage 132.0 (TID 4188) in 3 ms on 142a3f268980 (executor driver) (115/200)
24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_115 locally
24/09/02 14:27:39 INFO Executor: Finished task 115.0 in stage 132.0 (TID 4189). 5344 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 116.0 in stage 132.0 (TID 4190) (142a3f268980, executor driver, partition 116, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Fini

+-----------------+-----------+-----------+-----------------+----------------+----------------------+----------+
|        user_name|CreateEvent|DeleteEvent|IssueCommentEvent|PullRequestEvent|PullRequestReviewEvent|WatchEvent|
+-----------------+-----------+-----------+-----------------+----------------+----------------------+----------+
|leslie-fang-intel|       null|       null|                1|            null|                     2|      null|
|cybersecurity-dev|       null|       null|             null|            null|                  null|         1|
|          xuhancn|       null|       null|             null|               1|                  null|      null|
|    SherlockNoMad|       null|       null|             null|            null|                     1|      null|
+-----------------+-----------+-----------+-----------------+----------------+----------------------+----------+



24/09/02 14:27:39 INFO Executor: Finished task 186.0 in stage 132.0 (TID 4260). 5344 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 187.0 in stage 132.0 (TID 4261) (142a3f268980, executor driver, partition 187, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Finished task 186.0 in stage 132.0 (TID 4260) in 3 ms on 142a3f268980 (executor driver) (187/200)
24/09/02 14:27:39 INFO Executor: Running task 187.0 in stage 132.0 (TID 4261)
24/09/02 14:27:39 INFO BlockManager: Found block rdd_449_187 locally
24/09/02 14:27:39 INFO Executor: Finished task 187.0 in stage 132.0 (TID 4261). 5344 bytes result sent to driver
24/09/02 14:27:39 INFO TaskSetManager: Starting task 188.0 in stage 132.0 (TID 4262) (142a3f268980, executor driver, partition 188, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:27:39 INFO TaskSetManager: Finished task 187.0 in stage 132.0 (TID 4261) in 3 ms on 142a3f268980 (ex

In [8]:
pytorch_filter = PytorchTopIssuerFilter(args)
pytorch_df = pytorch_filter.filter(df)
if pytorch_df is not None:
    pytorch_df = df_with_meta(pytorch_df, args.target_date)
    pytorch_df.show(truncate=False, vertical=False)
    #es.write_df(pytorch_df, "pytorch-top-issuer")


+-----------------------------+-----------+-----------+---------+-----------------+-----------+----------------+-----------------------------+----------------------+---------+----------+----------+
|user_name                    |CreateEvent|DeleteEvent|ForkEvent|IssueCommentEvent|IssuesEvent|PullRequestEvent|PullRequestReviewCommentEvent|PullRequestReviewEvent|PushEvent|WatchEvent|@timestamp|
+-----------------------------+-----------+-----------+---------+-----------------+-----------+----------------+-----------------------------+----------------------+---------+----------+----------+
|SakuraMathcraft              |null       |null       |null     |null             |null       |null            |null                         |null                  |null     |1         |null      |
|briancoutinho                |null       |null       |null     |1                |null       |null            |null                         |null                  |null     |null      |null      |
|linux-fou

In [12]:
pytorch_df.show()

+--------------------+-----------+-----------+---------+-----------------+-----------+----------------+-----------------------------+----------------------+---------+----------+----------+
|           user_name|CreateEvent|DeleteEvent|ForkEvent|IssueCommentEvent|IssuesEvent|PullRequestEvent|PullRequestReviewCommentEvent|PullRequestReviewEvent|PushEvent|WatchEvent|@timestamp|
+--------------------+-----------+-----------+---------+-----------------+-----------+----------------+-----------------------------+----------------------+---------+----------+----------+
|     SakuraMathcraft|       null|       null|     null|             null|       null|            null|                         null|                  null|     null|         1|      null|
|       briancoutinho|       null|       null|     null|                1|       null|            null|                         null|                  null|     null|      null|      null|
|linux-foundation-...|       null|       null|     null

In [39]:
# top user filter
user_filter = TopUserFilter(args)
user_df = user_filter.filter(df)
user_df = df_with_meta(user_df, args.target_date)

24/09/02 14:29:13 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:29:13 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:29:13 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, type: string>
24/09/02 14:29:13 INFO BlockManagerInfo: Removed broadcast_160_piece0 on 142a3f268980:46091 in memory (size: 21.1 KiB, free: 1663.0 MiB)
24/09/02 14:29:13 INFO BlockManagerInfo: Removed broadcast_161_piece0 on 142a3f268980:46091 in memory (size: 20.9 KiB, free: 1663.1 MiB)
24/09/02 14:29:13 INFO CodeGenerator: Code generated in 17.8317 ms
24/09/02 14:29:13 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/09/02 14:29:13 INFO CodeGenerator: Code generated in 4.6234 

24/09/02 14:29:15 INFO TaskSetManager: Starting task 60.0 in stage 135.0 (TID 4350) (142a3f268980, executor driver, partition 60, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:15 INFO TaskSetManager: Finished task 59.0 in stage 135.0 (TID 4349) in 3 ms on 142a3f268980 (executor driver) (74/200)
24/09/02 14:29:15 INFO Executor: Running task 60.0 in stage 135.0 (TID 4350)
24/09/02 14:29:15 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:15 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:15 INFO Executor: Finished task 60.0 in stage 135.0 (TID 4350). 3699 bytes result sent to driver
24/09/02 14:29:15 INFO TaskSetManager: Starting task 61.0 in stage 135.0 (TID 4351) (142a3f268980, executor driver, partition 61, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:15 INFO TaskSetManager: Finished tas

24/09/02 14:29:16 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:16 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:16 INFO Executor: Finished task 151.0 in stage 135.0 (TID 4430). 3699 bytes result sent to driver
24/09/02 14:29:16 INFO TaskSetManager: Starting task 152.0 in stage 135.0 (TID 4431) (142a3f268980, executor driver, partition 152, PROCESS_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:16 INFO TaskSetManager: Finished task 151.0 in stage 135.0 (TID 4430) in 3 ms on 142a3f268980 (executor driver) (155/200)
24/09/02 14:29:16 INFO Executor: Running task 152.0 in stage 135.0 (TID 4431)
24/09/02 14:29:16 INFO ShuffleBlockFetcherIterator: Getting 0 (0.0 B) non-empty blocks including 0 (0.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:16 INFO ShuffleBlockFetcherIterator: Started 0 remo

In [40]:
user_df.show()

24/09/02 14:29:21 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:29:21 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#835),NOT (actor#835.login = github-actions[bot])
24/09/02 14:29:21 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, type: string>
24/09/02 14:29:21 INFO BlockManagerInfo: Removed broadcast_164_piece0 on 142a3f268980:46091 in memory (size: 14.7 KiB, free: 1663.0 MiB)
24/09/02 14:29:21 INFO BlockManagerInfo: Removed broadcast_165_piece0 on 142a3f268980:46091 in memory (size: 14.8 KiB, free: 1663.0 MiB)
24/09/02 14:29:21 INFO BlockManagerInfo: Removed broadcast_163_piece0 on 142a3f268980:46091 in memory (size: 11.4 KiB, free: 1663.0 MiB)
24/09/02 14:29:21 INFO CodeGenerator: Code generated in 17.1846 ms
24/09/02 14:29:21 INFO MemoryStore: Block broadcast_166 stored as values in memory (estimated size 176.3 KiB, fr

24/09/02 14:29:23 INFO Executor: Finished task 7.0 in stage 138.0 (TID 4485). 4383 bytes result sent to driver
24/09/02 14:29:23 INFO TaskSetManager: Starting task 8.0 in stage 138.0 (TID 4486) (142a3f268980, executor driver, partition 8, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:23 INFO TaskSetManager: Finished task 7.0 in stage 138.0 (TID 4485) in 19 ms on 142a3f268980 (executor driver) (8/200)
24/09/02 14:29:23 INFO Executor: Running task 8.0 in stage 138.0 (TID 4486)
24/09/02 14:29:23 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:23 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:23 INFO Executor: Finished task 8.0 in stage 138.0 (TID 4486). 4383 bytes result sent to driver
24/09/02 14:29:23 INFO TaskSetManager: Starting task 9.0 in stage 138.0 (TID 4487) (142a3f268980, executor driver, partition 9, NOD

24/09/02 14:29:23 INFO Executor: Finished task 28.0 in stage 138.0 (TID 4506). 4383 bytes result sent to driver
24/09/02 14:29:23 INFO TaskSetManager: Starting task 29.0 in stage 138.0 (TID 4507) (142a3f268980, executor driver, partition 29, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:23 INFO TaskSetManager: Finished task 28.0 in stage 138.0 (TID 4506) in 19 ms on 142a3f268980 (executor driver) (29/200)
24/09/02 14:29:23 INFO Executor: Running task 29.0 in stage 138.0 (TID 4507)
24/09/02 14:29:23 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:23 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:23 INFO Executor: Finished task 29.0 in stage 138.0 (TID 4507). 4383 bytes result sent to driver
24/09/02 14:29:23 INFO TaskSetManager: Starting task 30.0 in stage 138.0 (TID 4508) (142a3f268980, executor driver, partitio

24/09/02 14:29:24 INFO TaskSetManager: Starting task 40.0 in stage 138.0 (TID 4518) (142a3f268980, executor driver, partition 40, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:24 INFO TaskSetManager: Finished task 39.0 in stage 138.0 (TID 4517) in 18 ms on 142a3f268980 (executor driver) (40/200)
24/09/02 14:29:24 INFO Executor: Running task 40.0 in stage 138.0 (TID 4518)
24/09/02 14:29:24 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:24 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:24 INFO Executor: Finished task 40.0 in stage 138.0 (TID 4518). 4383 bytes result sent to driver
24/09/02 14:29:24 INFO TaskSetManager: Starting task 41.0 in stage 138.0 (TID 4519) (142a3f268980, executor driver, partition 41, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:24 INFO TaskSetManager: Finished task

24/09/02 14:29:24 INFO Executor: Finished task 60.0 in stage 138.0 (TID 4538). 4383 bytes result sent to driver
24/09/02 14:29:24 INFO TaskSetManager: Starting task 61.0 in stage 138.0 (TID 4539) (142a3f268980, executor driver, partition 61, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:24 INFO TaskSetManager: Finished task 60.0 in stage 138.0 (TID 4538) in 19 ms on 142a3f268980 (executor driver) (61/200)
24/09/02 14:29:24 INFO Executor: Running task 61.0 in stage 138.0 (TID 4539)
24/09/02 14:29:24 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:24 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:24 INFO Executor: Finished task 61.0 in stage 138.0 (TID 4539). 4383 bytes result sent to driver
24/09/02 14:29:24 INFO TaskSetManager: Starting task 62.0 in stage 138.0 (TID 4540) (142a3f268980, executor driver, partitio

...

24/09/02 14:29:26 INFO Executor: Finished task 164.0 in stage 138.0 (TID 4642). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 165.0 in stage 138.0 (TID 4643) (142a3f268980, executor driver, partition 165, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:26 INFO TaskSetManager: Finished task 164.0 in stage 138.0 (TID 4642) in 19 ms on 142a3f268980 (executor driver) (165/200)
24/09/02 14:29:26 INFO Executor: Running task 165.0 in stage 138.0 (TID 4643)
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Getting 1 (6.3 KiB) non-empty blocks including 1 (6.3 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:26 INFO Executor: Finished task 165.0 in stage 138.0 (TID 4643). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 166.0 in stage 138.0 (TID 4644) (142a3f268980, executor driver, 

24/09/02 14:29:26 INFO Executor: Finished task 176.0 in stage 138.0 (TID 4654). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 177.0 in stage 138.0 (TID 4655) (142a3f268980, executor driver, partition 177, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:26 INFO TaskSetManager: Finished task 176.0 in stage 138.0 (TID 4654) in 18 ms on 142a3f268980 (executor driver) (177/200)
24/09/02 14:29:26 INFO Executor: Running task 177.0 in stage 138.0 (TID 4655)
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:26 INFO Executor: Finished task 177.0 in stage 138.0 (TID 4655). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 178.0 in stage 138.0 (TID 4656) (142a3f268980, executor driver, 

24/09/02 14:29:26 INFO Executor: Finished task 187.0 in stage 138.0 (TID 4665). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 188.0 in stage 138.0 (TID 4666) (142a3f268980, executor driver, partition 188, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:26 INFO TaskSetManager: Finished task 187.0 in stage 138.0 (TID 4665) in 17 ms on 142a3f268980 (executor driver) (188/200)
24/09/02 14:29:26 INFO Executor: Running task 188.0 in stage 138.0 (TID 4666)
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:26 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:26 INFO Executor: Finished task 188.0 in stage 138.0 (TID 4666). 4383 bytes result sent to driver
24/09/02 14:29:26 INFO TaskSetManager: Starting task 189.0 in stage 138.0 (TID 4667) (142a3f268980, executor driver, 

+--------------------+------------------+-----------+-----------+---------+-----------+-----------------+-----------+-----------+-----------+----------------+-----------------------------+----------------------+---------+------------+----------+----------+
|           user_name|CommitCommentEvent|CreateEvent|DeleteEvent|ForkEvent|GollumEvent|IssueCommentEvent|IssuesEvent|MemberEvent|PublicEvent|PullRequestEvent|PullRequestReviewCommentEvent|PullRequestReviewEvent|PushEvent|ReleaseEvent|WatchEvent|@timestamp|
+--------------------+------------------+-----------+-----------+---------+-----------+-----------------+-----------+-----------+-----------+----------------+-----------------------------+----------------------+---------+------------+----------+----------+
|             oxdev03|              null|       null|       null|     null|       null|             null|       null|       null|       null|               1|                         null|                  null|        1|        

24/09/02 14:29:27 INFO Executor: Finished task 198.0 in stage 138.0 (TID 4676). 4383 bytes result sent to driver
24/09/02 14:29:27 INFO TaskSetManager: Starting task 199.0 in stage 138.0 (TID 4677) (142a3f268980, executor driver, partition 199, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:29:27 INFO TaskSetManager: Finished task 198.0 in stage 138.0 (TID 4676) in 18 ms on 142a3f268980 (executor driver) (199/200)
24/09/02 14:29:27 INFO Executor: Running task 199.0 in stage 138.0 (TID 4677)
24/09/02 14:29:27 INFO ShuffleBlockFetcherIterator: Getting 1 (6.9 KiB) non-empty blocks including 1 (6.9 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:29:27 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:29:27 INFO Executor: Finished task 199.0 in stage 138.0 (TID 4677). 4383 bytes result sent to driver
24/09/02 14:29:27 INFO TaskSetManager: Finished task 199.0 in stage 138.0 (TID 4677) in 17 ms on 142a3f268980 (execut

In [27]:
# store data to ES
es = Es("http://es:9200")
# es.write_df(stat_df, "daily-stats-2024")
# es.write_df(repo_df, "top-repo-2024")
# es.write_df(user_df, "top-user-2024")

24/09/02 14:18:22 INFO InMemoryFileIndex: It took 31 ms to list leaf files for 1 paths.
24/09/02 14:18:22 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/02 14:18:23 INFO FileSourceStrategy: Pushed Filters: 
24/09/02 14:18:23 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/02 14:18:23 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
24/09/02 14:18:23 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 176.4 KiB, free 1663.0 MiB)
24/09/02 14:18:23 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 28.1 KiB, free 1663.0 MiB)
24/09/02 14:18:23 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 142a3f268980:46091 (size: 28.1 KiB, free: 1663.2 MiB)
24/09/02 14:18:23 INFO SparkContext: Created broadcast 0 from json at NativeMethodAccessorImpl.java:0
24/09/02 14:18:23 INFO FileSourceScanExec: Planning scan with bin packing, max size: 60559581 bytes, open cost is considered as

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/09/02 14:18:28 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:18:28 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#7),NOT (actor#7.login = github-actions[bot])
24/09/02 14:18:28 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string, gravatar_id: string, id: bigint, login: string ... 1 more field>, type: string>
24/09/02 14:18:28 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 142a3f268980:46091 in memory (size: 28.1 KiB, free: 1663.2 MiB)
24/09/02 14:18:28 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 142a3f268980:46091 in memory (size: 6.6 KiB, free: 1663.2 MiB)
24/09/02 14:18:28 INFO FileSourceStrategy: Pushed Filters: IsNotNull(actor)
24/09/02 14:18:28 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(actor#7),NOT (actor#7.login = github-actions[bot])
24/09/02 14:18:28 INFO FileSourceStrategy: Output Data Schema: struct<actor: struct<avatar_url: string, display_login: string,

24/09/02 14:18:31 INFO Executor: Finished task 0.0 in stage 1.0 (TID 1). 2027 bytes result sent to driver
24/09/02 14:18:31 INFO TaskSetManager: Starting task 0.0 in stage 3.0 (TID 2) (142a3f268980, executor driver, partition 0, PROCESS_LOCAL, 4871 bytes) taskResourceAssignments Map()
24/09/02 14:18:31 INFO Executor: Running task 0.0 in stage 3.0 (TID 2)
24/09/02 14:18:31 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 1974 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:18:31 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/09/02 14:18:31 INFO DAGScheduler: ShuffleMapStage 1 ($anonfun$withThreadLocalCaptured$1 at FutureTask.java:264) finished in 1.985 s
24/09/02 14:18:31 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:18:31 INFO DAGScheduler: running: Set(ShuffleMapStage 9, ShuffleMapStage 5, ShuffleMapStage 3, ShuffleMapStage 7)
24/09/02 14:18:31 INFO DAGScheduler: waiting: Set(ResultStage 2, ResultStage

24/09/02 14:18:33 INFO BlockManagerInfo: Removed broadcast_13_piece0 on 142a3f268980:46091 in memory (size: 8.8 KiB, free: 1663.0 MiB)
24/09/02 14:18:33 INFO BlockManagerInfo: Removed broadcast_8_piece0 on 142a3f268980:46091 in memory (size: 8.6 KiB, free: 1663.0 MiB)
24/09/02 14:18:33 INFO BlockManagerInfo: Removed broadcast_12_piece0 on 142a3f268980:46091 in memory (size: 8.8 KiB, free: 1663.0 MiB)
24/09/02 14:18:35 INFO Executor: Finished task 0.0 in stage 5.0 (TID 5). 2027 bytes result sent to driver
24/09/02 14:18:35 INFO TaskSetManager: Starting task 0.0 in stage 7.0 (TID 6) (142a3f268980, executor driver, partition 0, PROCESS_LOCAL, 4871 bytes) taskResourceAssignments Map()
24/09/02 14:18:35 INFO TaskSetManager: Finished task 0.0 in stage 5.0 (TID 5) in 1663 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:18:35 INFO Executor: Running task 0.0 in stage 7.0 (TID 6)
24/09/02 14:18:35 INFO TaskSchedulerImpl: Removed TaskSet 5.0, whose tasks have all completed, from pool 
24/0

...

24/09/02 14:19:05 INFO BlockManagerInfo: Removed broadcast_72_piece0 on 142a3f268980:46091 in memory (size: 5.0 KiB, free: 1663.0 MiB)
24/09/02 14:19:05 INFO BlockManagerInfo: Removed broadcast_71_piece0 on 142a3f268980:46091 in memory (size: 27.3 KiB, free: 1663.0 MiB)
24/09/02 14:19:06 INFO Executor: Finished task 0.0 in stage 54.0 (TID 1435). 2741 bytes result sent to driver
24/09/02 14:19:06 INFO TaskSetManager: Finished task 0.0 in stage 54.0 (TID 1435) in 1770 ms on 142a3f268980 (executor driver) (1/1)
24/09/02 14:19:06 INFO TaskSchedulerImpl: Removed TaskSet 54.0, whose tasks have all completed, from pool 
24/09/02 14:19:06 INFO DAGScheduler: ShuffleMapStage 54 (showString at NativeMethodAccessorImpl.java:0) finished in 1.773 s
24/09/02 14:19:06 INFO DAGScheduler: looking for newly runnable stages
24/09/02 14:19:06 INFO DAGScheduler: running: Set()
24/09/02 14:19:06 INFO DAGScheduler: waiting: Set(ResultStage 56, ShuffleMapStage 55)
24/09/02 14:19:06 INFO DAGScheduler: failed: S

24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:06 INFO Executor: Finished task 49.0 in stage 55.0 (TID 1485). 4181 bytes result sent to driver
24/09/02 14:19:06 INFO TaskSetManager: Starting task 50.0 in stage 55.0 (TID 1486) (142a3f268980, executor driver, partition 50, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:06 INFO TaskSetManager: Finished task 49.0 in stage 55.0 (TID 1485) in 4 ms on 142a3f268980 (executor driver) (50/200)
24/09/02 14:19:06 INFO Executor: Running task 50.0 in stage 55.0 (TID 1486)
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Getting 1 (3.5 KiB) non-empty blocks including 1 (3.5 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:06 INFO Executor: Finished task 50.0 in stage 55.0 (TID 1486). 4181 bytes result sent to driver
24/09/02 14:19:06 INFO TaskSetManager: Starti

24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Getting 1 (3.2 KiB) non-empty blocks including 1 (3.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:06 INFO Executor: Finished task 99.0 in stage 55.0 (TID 1535). 4181 bytes result sent to driver
24/09/02 14:19:06 INFO TaskSetManager: Starting task 100.0 in stage 55.0 (TID 1536) (142a3f268980, executor driver, partition 100, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:06 INFO TaskSetManager: Finished task 99.0 in stage 55.0 (TID 1535) in 4 ms on 142a3f268980 (executor driver) (100/200)
24/09/02 14:19:06 INFO Executor: Running task 100.0 in stage 55.0 (TID 1536)
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Getting 1 (3.2 KiB) non-empty blocks including 1 (3.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 remot

+------------+------------+----------+--------+----------+--------------------+----------+
|d_user_count|d_repo_count|push_count|pr_count|fork_count|commit_comment_count|@timestamp|
+------------+------------+----------+--------+----------+--------------------+----------+
|       37540|       27844|     74882|    7573|      1685|                  89|2024-09-01|
+------------+------------+----------+--------+----------+--------------------+----------+



24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Getting 1 (3.5 KiB) non-empty blocks including 1 (3.5 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
24/09/02 14:19:06 INFO Executor: Finished task 156.0 in stage 55.0 (TID 1592). 4181 bytes result sent to driver
24/09/02 14:19:06 INFO TaskSetManager: Starting task 157.0 in stage 55.0 (TID 1593) (142a3f268980, executor driver, partition 157, NODE_LOCAL, 4442 bytes) taskResourceAssignments Map()
24/09/02 14:19:06 INFO TaskSetManager: Finished task 156.0 in stage 55.0 (TID 1592) in 3 ms on 142a3f268980 (executor driver) (157/200)
24/09/02 14:19:06 INFO Executor: Running task 157.0 in stage 55.0 (TID 1593)
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Getting 1 (3.5 KiB) non-empty blocks including 1 (3.5 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) remote blocks
24/09/02 14:19:06 INFO ShuffleBlockFetcherIterator: Started 0 rem

## spark dataframe 을 elasticsearch 에 저장해보기


## 데이터 저장 로직을 구현한 elasticsearch.py 제출